# MNIST Inpainting

In [ ]:
# Install imgeio in order to generate an animated gif showing the image generating process
!pip install imageio

### Import TensorFlow and enable eager execution

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time

from IPython import display

### Load the dataset

We are going to use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
print(train_images.shape)

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

# TODO maybe shuffle
train_full_images = train_images[:len(train_images) // 2,:,:,:]
train_masked_images = train_images[len(train_images) // 2:,:,:,:]

print(train_full_images.shape)

In [ ]:
BUFFER_SIZE = 30000
BATCH_SIZE = 256

### Use tf.data to create batches and shuffle the dataset

In [ ]:
def mask(image):
  upper_edge = tf.ones([10,28,1], tf.float32)
  lower_edge = tf.ones([11,28,1], tf.float32)
  
  middle_left = tf.ones([7,10,1], tf.float32)
  middle_right = tf.ones([7,11,1], tf.float32)
  
  zeros = tf.zeros([7,7,1], tf.float32)
  
  middle = tf.concat([middle_left, zeros, middle_right], axis=1)
  mask = tf.concat([upper_edge, middle, lower_edge], axis=0)
  
  return image * mask
  

In [ ]:
train_full_images = tf.data.Dataset.from_tensor_slices(train_full_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train_unmasked_images = tf.data.Dataset.from_tensor_slices(train_masked_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_masked_images = train_unmasked_images.map(mask)

train_dataset = tf.data.Dataset.zip((train_full_images, train_masked_images, train_unmasked_images))


In [ ]:
i = 0
for image in train_masked_images:
  if i > 0:
    break
  print(image.shape)
  plt.imshow(image[0,:,:,0] * 127.5 + 127.5, cmap='gray')
  plt.colorbar()
  plt.axis('off')
  plt.show()
  i = i + 1


## Create the models

We will use tf.keras [Sequential API](https://www.tensorflow.org/guide/keras#sequential_model) to define the generator and discriminator models.

### The Generator Model

The generator is responsible for creating convincing images that are good enough to fool the discriminator. The network architecture for the generator consists of [Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose) (Upsampling) layers. We start with a fully connected layer and upsample the image two times in order to reach the desired image size of 28x28x1. We increase the width and height, and reduce the depth as we move through the layers in the network. We use [Leaky ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LeakyReLU) activation for each layer except for the last one where we use a tanh activation.

In [ ]:
def patch_image(patch, image):
  #TODO: Ver si se puede usar algo mejor
  
  upper_edge = image[:, :10, :, :]
  lower_edge = image[:, 17:, :, :]
  
  middle_left = image[:, 10:17, :10, :]
  middle_right = image[:, 10:17, 17:, :]
  
  middle = tf.concat([middle_left, patch, middle_right], axis=2)
  return tf.concat([upper_edge, middle, lower_edge], axis=1)


In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    
    # Encoder
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), padding='same', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    assert model.output_shape == (None, 28, 28, 64)
      
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    assert model.output_shape == (None, 14, 14, 128)
          
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    assert model.output_shape == (None, 7, 7, 256)

    # Decoder  
    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)  
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 64)    
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 7, 7, 1)
    
    #TODO concatenar patch con imagen aca con custom layer
  
    return model

### The Discriminator model

The discriminator is responsible for distinguishing fake images from real images. It's similar to a regular CNN-based image classifier.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
      
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
       
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
     
    return model

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

## Define the loss functions and the optimizer

Let's define the loss functions and the optimizers for the generator and the discriminator.


### Generator loss
The generator loss is a sigmoid cross entropy loss of the generated images and an array of ones, since the generator is trying to generate fake images that resemble the real images.

In [ ]:
def generator_loss(generated_output):
    return tf.losses.sigmoid_cross_entropy(tf.ones_like(generated_output), generated_output)

### Discriminator loss

The discriminator loss function takes two inputs: real images, and generated images. Here is how to calculate the discriminator loss:
1. Calculate real_loss which is a sigmoid cross entropy loss of the real images and an array of ones (since these are the real images).
2. Calculate generated_loss which is a sigmoid cross entropy loss of the generated images and an array of zeros (since these are the fake images).
3. Calculate the total_loss as the sum of real_loss and generated_loss.

In [ ]:
def discriminator_loss(real_output, generated_output):
    # [1,1,...,1] with real output since it is true and we want our generated examples to look like it
    real_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.ones_like(real_output), logits=real_output)

    # [0,0,...,0] with generated images since they are fake
    generated_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.zeros_like(generated_output), logits=generated_output)

    total_loss = real_loss + generated_loss

    return total_loss

The discriminator and the generator optimizers are different since we will train two networks separately.

In [ ]:
generator_optimizer = tf.train.AdamOptimizer(1e-4)
discriminator_optimizer = tf.train.AdamOptimizer(1e-4)

**Checkpoints (Object-based saving)**

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Set up GANs for Training



Now it's time to put together the generator and discriminator to set up the Generative Adversarial Networks, as you see in the diagam at the beginning of the tutorial.

**Define training parameters**

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# We'll re-use this masked image for the generator so
# it will be easier to see the improvement over time.

iterator = train_masked_images.make_one_shot_iterator()
masked_image_batch = iterator.get_next()
test_masked_images = masked_image_batch[:num_examples_to_generate, :, :, :]
print(test_masked_images.shape)

**Define training method**

We start by iterating over the dataset. The generator is given a random vector as an input which is processed to  output an image looking like a handwritten digit. The discriminator is then shown the real MNIST images as well as the generated images.

Next, we calculate the generator and the discriminator loss. Then, we calculate the gradients of loss with respect to both the generator and the discriminator variables.

In [ ]:
def train_step(full_images, masked_images):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_patches = generator(masked_images, training=True)
    
    generated_images = patch_image(generated_patches, masked_images)
    
    real_output = discriminator(full_images, training=True)
    generated_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(generated_output)
    disc_loss = discriminator_loss(real_output, generated_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.variables))


This model takes about ~30 seconds per epoch to train on a single Tesla K80 on Colab, as of October 2018. 

Eager execution can be slower than executing the equivalent graph as it can't benefit from whole-program optimizations on the graph, and also incurs overheads of interpreting Python code. By using [tf.contrib.eager.defun](https://www.tensorflow.org/api_docs/python/tf/contrib/eager/defun) to create graph functions, we get a ~20 secs/epoch performance boost (from ~50 secs/epoch down to ~30 secs/epoch). This way we get the best of both eager execution (easier for debugging) and graph mode (better performance).

In [ ]:
train_step = tf.contrib.eager.defun(train_step)

In [ ]:
def train(dataset, epochs):  
  for epoch in range(epochs):
    start = time.time()
    
    for images in dataset:
      full_images = images[0]
      masked_images = images[1]
      train_step(full_images, masked_images)

    display.clear_output(wait=True)
    generate_and_save_images(generator,
                               epoch + 1,
                               test_masked_images)
    
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print ('Time taken for epoch {} is {} sec'.format(epoch + 1,
                                                      time.time()-start))
  # generating after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           test_masked_images)

**Generate and save images**



In [ ]:
def generate_and_save_images(generator, epoch, masked_images):
  # make sure the training parameter is set to False because we
  # don't want to train the batchnorm layer when doing inference.
  patches = generator(masked_images, training=False)
  generated_images = patch_image(patches, masked_images)

  fig = plt.figure(figsize=(4,4))
  
  for i in range(generated_images.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(generated_images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')
        
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## Train the GANs
We will call the train() method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, you can see the generated digits look increasingly real. After 50 epochs, they look very much like the MNIST digits.

In [ ]:
%%time
train(train_dataset, EPOCHS)

**Restore the latest checkpoint**

In [1]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

NameError: name 'checkpoint' is not defined

In [ ]:
generate_and_save_images(generator,
                           EPOCHS + 1,
                           test_masked_images)

## Generated images 



After training, its time to generate some images! 
The last step is to plot the generated images and voila!


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

**Generate a GIF of all the saved images**

We will use imageio to create an animated gif using all the images saved during training.

In [ ]:
with imageio.get_writer('dcgan.gif', mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# this is a hack to display the gif inside the notebook
os.system('cp dcgan.gif dcgan.gif.png')

Display the animated gif with all the mages generated during the training of GANs.

In [ ]:
display.Image(filename="dcgan.gif.png")

**Download the animated gif**

Uncomment the code below to download an animated gif from Colab.

In [ ]:
#from google.colab import files
#files.download('dcgan.gif')

## Learn more about GANs


We hope this tutorial was helpful! As a next step, you might like to experiment with a different dataset, for example the Large-scale Celeb Faces Attributes (CelebA) dataset [available on Kaggle](https://www.kaggle.com/jessicali9530/celeba-dataset/home).

To learn more about GANs:

* Check out MIT's lecture (linked above), or [this](http://cs231n.stanford.edu/slides/2018/cs231n_2018_lecture12.pdf) lecture form Stanford's CS231n. 

* We also recommend the [CVPR 2018 Tutorial on GANs](https://sites.google.com/view/cvpr2018tutorialongans/), and the [NIPS 2016 Tutorial: Generative Adversarial Networks](https://arxiv.org/abs/1701.00160).
